# CSE204: Lab exam session, 03/13/2019

@author: Theo Lacombe

Please read the following carefully before starting the exam.

## Overall presentation

This lab exam is composed of 3 exercises, each of them granting up to 7 points (the final grade is over 20, so there is one bonus point).

These exercises are independant.

## Rules and other remarks

Obviously, standard exam rules hold for this lab session, that is no chatting, no use of communication tools, no use of smartphone, etc.

As this is a exam on computer, some details must be specified:
- You are allowed to call the responsible for **technical** (i.e. Python related) help in order to understand an error message if needed (that does not mean that we will debug your code).
- Your answers to written questions (like "What do you observe?") must be **short**, basically one sentence at most (most of the time, a single word would fit perfectly).
- You are allowed to use all standard Python tools along with the `numpy` library. However, you are **not** allowed to use `scikit-learn` except when it is explicitely mentionned in the question. (Basically, it means that if the question is "implement a linear regression", you can't just import the one of `scikit-learn`.)
- Use `np.array`, not `np.matrix`.
- You are not allowed to use any complementary document.
- You won't be evaluated on the quality of your code, only on its correctness.
- You must not change the name of the functions pre-written.
- Beware of the cells you run (and the order you run them): don't train the algorithm of exercise 3 on the data of exercise 2 by mistake!

## Imports

In [ ]:
%matplotlib inline
import numpy as np

from sklearn.neighbors import KNeighborsClassifier

# a side file that contains some useful test functions & cie.
from utils import *

# Exercise 1 : $k$-nearest neighbors classifier

We briefly recall the $k$ nearest neighbors ($k$-NN) algorithm seen in lab session 2, which is used as a classification method here.

Consider a dataset $\mathcal{D} = (X, y)$ where the $k$-th coordinate of the $i$-th observation is denoted by $x^{(i)}_k$ and the corresponding label is $y^{(i)}$, or equilvalently:
$$\boldsymbol{X} = \begin{pmatrix} x_1^{(1)} & \dots & x_p^{(1)} \\ \vdots & \dots & \vdots \\ x_1^{(n)} & \dots & x_p^{(n)} \end{pmatrix}
\quad \quad
\boldsymbol{y} = \begin{pmatrix} y^{(1)} \\ \vdots \\ y^{(n)} \end{pmatrix}.$$

Assume a fixed integer $k < n$. Given a new observation $x \in \mathbb{R}^p$, we produce an estimation of its class $\hat{y}$ in the following way:

Let $x^{(i_1)}, \dots, x^{(i_k)}$ be the $k$ nearest neighbors of $x$ in $X$ (i.e. the points in $X$ that achieve the $k$ lowest values of $\|x - x^{(i)}\|_2$). Then $\hat{y}$ is the predominent occurence in $\{ y^{(i_1)}, \dots, y^{(i_k)} \}$. In case of equality between two (or more) classes, $\hat{y}$ is randomly chosen (uniformly) among these classes. 

**Question 1:** What is the training error achieved by this algorithm in the case $k=1$?

<-- write your answer here -->

**Question 2:** Is the $k$-NN algorithm a _parametric_ or _non-parametric_ model?

<-- write your answer here -->

**Question 3:** Implement a function `oneNN` that takes as an input a matrix `X_train` of shape `n x p`, corresponding labels `y_train` (shape `n`) a new observation `x` of shape `p`, and that returns an estimation of `hat_y` for this new observation using the **one**-nearest neighbor algorithm.

_Python hint:_ You can compute the distance between two numpy arrays `x_i` and `x` using `np.linalg.norm(x_i - x)`.

In [ ]:
def oneNN(X_train, y_train, x):
    """
        param X_train: np.array, shape N x p
        param y_train: np.array, shape N
        param x:       np.array, shape p
        return: hat_y, class prediction.
    """
    return None #<-- to complete

Test your function by running the following cell. Running time should be about few seconds. 
Note that the grid used to plot classification boundaries is intentionally coarse (to avoid running time issues).

In [ ]:
test_oneNN(oneNN)

We consider a dataset of $n = 700$ points, with $3$ labels "blue", "red", and "green", encoded respectively by $0,1,2$ in the following. This dataset is splitted in a training set `x_train, y_train` with $300$ observations, and a test set `x_test, y_test` of $400$ points ; this dataset must be loaded by running the following cell.

In [ ]:
x_train, y_train, x_test, y_test = load_kNN_dataset()

**Question 4:** In this question, we allow the use of the `scikit-learn` library. 

**4.a.** Using `scikit-learn`, perform $k$-NN classification for all $k = 1 \dots 10$ by training on (`x_train`, `y_train`), and use it to predict labels for `x_test`. Evaluate the quality of the classification using the `accuracy_score` function provided by `scikit-learn`.


We recall the use of `scikit-learn` to perform $k$-NN classification:
- `clf = KNeighborsClassifier(n_neighbors = k)` creates an object of the class `classifier` that can be used to perform `n_neighbors`-NN classification (of course, you have to precise the value of `k` when instanciating the classifier).
- `clf.fit(x_train,y_train)` trains this classifier on a training set (`x_train` is the observation matrix, `y_train` the labels).
- `clf.predict(x_test)` returns label estimations for a new set of observations encoded as a matrix `x_test`.
- Given `y_test` and `y_pred` which are respectively the true labels of the test set and the label predicted by the $k$-NN classifier, you can compute the accuracy score (in %) of the classification using `score(y_test, y_pred)`.

In [ ]:
# write some code here to answer the question

**4.b.** Which value of $k$ reaches the best accuracy on the test set?

<-- write your answer here-->

# Exercise 2 : Linear regression and its extensions

We briefly recall the framework of linear regression.

Consider observations $x = x^{(1)} \dots x^{(n)}$, with $x^{(i)} \in \mathbb{R}^p$, and labels $y^{(1)} \dots y^{(n)}$.
Given an observation $x^{(i)}$, and a vector $\theta = (\theta_0 \dots \theta_p)^T$, we produce an estimation $\hat{y}^{(i)}$ of $y^{(i)}$ of the following form:
$$ \hat{y}^{(i)} = \theta_0 + \theta_1 x^{(i)}_1 + \dots + \theta_p x^{(i)}_p = \theta_0 + x^{(i)} \cdot (\theta_1 , \dots , \theta_p)^T.$$
We want to find an optimal vector $\theta^*$ so that the average error made by using $\hat{y}^{(i)}$ to approximate $y^{(i)}$ is small.
With vector notations, it reads:
$$\theta^* \in \mathrm{argmin} (E(\theta)), \quad \text{ where } \quad E(\theta) := \frac{1}{n} \| y - X \cdot \theta\|_2, $$
where 
$$
\boldsymbol{X} = \begin{pmatrix} 1 & x_1^{(1)} & \dots & x_p^{(1)} \\ \vdots & \vdots & \dots & \vdots \\ 1 & x_1^{(n)} & \dots & x_p^{(n)} \end{pmatrix}
\quad \quad
\boldsymbol{y} = \begin{pmatrix} y^{(1)} \\ \vdots \\ y^{(n)} \end{pmatrix}.
$$
We recall that a solution in closed form exists, namely the optimal $\theta^*$ is given by:
$$ \theta^* = (X^T X)^{-1} X^T y$$

Some Python hints useful in this exercise:
_Python hints_ : 
- You can use `np.linalg.inv(A)` to compute the inverse of a (non-singular) **square** matrix. 
- You can use `np.transpose(A)` or equivalently `A.T` to compute the transpose of a numpy array `A`. 
- You can use `np.dot(A,B)`, or equivalently `A.dot(B)` to compute the matrix-matrix (or matrix-vector) $A \cdot B$ product between two numpy arrays `A` and `B`. Beware of the shapes!
- Given a 1D np.array (encoding a vector) `t=[t_0 ... t_(n-1)]`, using `t[i:]` will return `[t_i, ... , t_(n-1)]`.

## Vanilla linear regression

**Question 1:** Implement a function `linreg` that, given `X` and `y`, returns the vector optimal `theta`.

In [ ]:
def linreg(X,y):
    """
        param X: np.array of shape N x (p+1)
        param y: np.array of shape N
        return:  np.array of shape (p+1)
    """
    return None #<-- to complete

**Question 2:** Implement a function `E` that, given `X,y,theta` computes the error $E(\theta)$.

_Hint:_ Do not forget the $\frac{1}{n}$ factor.

_Python hint:_ You can compute the distance $\| \cdot \|$ between two vector `hat_y` and `y` using `np.linalg.norm(hat_y - y)`.

In [ ]:
def E(X,y,theta):
    """
        param X:     np.array of shape N x (p+1)
        param y:     np.array of shape N
        param theta: np.array of shape (p+1)
        return: float, evaluation of the error function.
    """
    return None #<-- to complete

**Question 3:** Implement a function `pred` that, given an observation `x` of shape `p` and a vector `theta`, returns a predicted value `hat_y`.

_Hint:_ Do not forget the constant term $\theta_0$.

In [ ]:
def pred(x,theta):
    """
        param x:     np.array of shape p
        param theta: np.array of shape (p+1)
    """
    return None #<-- to complete

Test your functions with the following script.

In [ ]:
test_linear_regression(linreg, E, pred)

## Polynomial regression

We consider the specific case where the observations $x^{(i)}$ are real-valued (that is $p=1$). For an integer $k$ fixed, and a vector $\theta = (\theta_0 \dots \theta_k)^T$, we propose to estimate $y^{(i)}$ in the following way:
$$ \hat{y}^{(i)} = \theta_0 + \theta_1 x^{(i)} + \theta_2 (x^{(i)})^2 + \dots + \theta_k (x^{(i)})^k. $$
Here, $(x^{(i)})^j$ means the $j$-th power of $x^{(i)}$.

As in standard linear regression, the goal is to find a $\theta^*$ so that 
$$ \frac{1}{n} \|y - \hat{y}\|_2 $$
is minimal.

**Question 4:** Write a function `polynomial_expend` that, given a **real valued** vector $x \in \mathbb{R}^n$ and an integer $k$, returns the $n \times (k+1)$ matrix
$$X = \begin{pmatrix} 1 & x^{(1)} & \dots & (x^{(1)})^k \\ \vdots & \vdots & \dots & \vdots \\ 1 & x^{(n)} & \dots & (x^{(n)})^k \end{pmatrix}$$

_Python hint_: Given a vector `x`, `x**k` returns the vector whose coordinates are the $p$-th power of those in `x`.

In [ ]:
def polynomial_expend(x,k):
    """
        param x: np.array of shape N
        param k: integer
        return:  np.array of shape N x (k+1)
    """
    return None #<--to complete

We consider a dataset of $n = 100$ couple of (observations, labels), that is splitted into a _train set_ of size $50$ and a _test set_ of size $50$. These data are loaded using the following cell.

In [ ]:
x_train, y_train, x_test, y_test = load_data_polyreg()

**Question 5:** Using the functions that you have written above, 

**5.a.** Perform a polynomial regression to fit `x_train, y_train` ; with $k = 1 \dots 12$ and predict labels for the test set `x_test`. 

The test error is defined as
$$ E_\mathrm{test} (\hat{y}, y_\mathrm{test}) = \frac{1}{n_\mathrm{test}} \|\hat{y} - y_\mathrm{test}\|_2, $$
where $n_\mathrm{test}$ is the number of observation in the test set.

Store training errors and test errors you obtain as two `np.array`s of size $12$, whose names must be (respectively) `train_errors` and `test_errors`, and such that `train_errors[k]` (resp. `test_errors[k]`) gives the training error (resp. test error) obtained by performing polynomial regression with degree $k$.

_Python hint:_ Recall that you can compute the distance between two numpy array using `np.linalg.norm(hat_y, y_test)`.

In [ ]:
#write some code here to obtain the two list `train_scores` and `test_scores`.

#...

#train_errors = ... #<--to uncomment and to complete
#test_errors = ... #<--to uncomment and to complete

Run the following cell (beware of variable names!)

In [ ]:
plot_linreg_errors(train_errors, test_errors)

**5.b.** What is the name of the phenomenon we are observing here? (short answer)

<-- write something here -->

# Exercice 3 : the logistic regression

The logistic regression is a common model used to perform binary classification that is however formulated as a regression problem. 

We have a *learning set* $\mathcal{D} = \{{x^{(i)}}, y^{(i)}\}_{1 \leq i \leq n}$, where $y_i \in \{0,1\}$ and $x_i \in \mathbb{R}^p$.

We consider the _sigmoid_ (logistic) function:
$$ \sigma : t \mapsto \frac{1}{1 + e^{-t}}.$$
Note that $\sigma$ takes values in $(0,1)$ and will be used to estimate $y^{(i)}$ given $x^{(i)}$. More precisely, given a weight vector $\theta = ( \theta_0 \dots \theta_p )^T$ and a vector $X^{(i)} = (1, x^{(i)}_1 \dots x^{(i)}_p)$, we make the following estimation:
$$
    \hat{y}^{(i)} = \begin{cases} 1 \text{ if } \sigma(X^{(i)} \theta) > 1/2 \\
                                  0 \text{ otherwise } 
                    \end{cases}
$$

We want to find a good $\theta$. One can show that the optimal $\boldsymbol{\theta}^* = \begin{pmatrix} \theta_0^* & \dots & \theta_p^* \end{pmatrix}^T$ minimizes the following optimization problem:

$$\boldsymbol{\theta}^* \leftarrow \arg\min_{\boldsymbol{\theta}} E(\boldsymbol{\theta})$$
where
$$E(\boldsymbol{\theta}) = - \frac{1}{n} \sum_{i=1}^n \log(\sigma(X^{(i)} \cdot \theta)) y^{(i)} + \log(1 - \sigma(X^{(i)} \cdot \theta)) (1 - y^{(i)})$$
with

$$
\boldsymbol{X} = \begin{pmatrix} 1 & x_1^{(1)} & \dots & x_p^{(1)} \\ \vdots & \vdots & \dots & \vdots \\ 1 & x_1^{(n)} & \dots & x_p^{(n)} \end{pmatrix}
\quad \quad
\boldsymbol{y} = \begin{pmatrix} y^{(1)} \\ \vdots \\ y^{(n)} \end{pmatrix}
\quad \quad
\boldsymbol{\theta} = \begin{pmatrix} \theta_0 \\ \vdots \\ \theta_p \end{pmatrix}
$$

**Question 1** : Does the error function $E$ have a unique minimum?

<-- answer here -->

**Question 2:** 

**2.a.** Implement a function `my_dot` that computes, given $x^{(i)} \in \mathbb{R}^p$ and $\theta \in \mathbb{R}^{p+1}$ the quantity $X^{(i)} \cdot \theta = \theta_0 + x^{(i)} \cdot (\theta_1 , \dots , \theta_p)$.

In [ ]:
def my_dot(x,theta):
    return None #<-- to complete

**2.b.** Implement a function `sigmoid` that computes, given given $x^{(i)} \in \mathbb{R}^p$ and $\theta \in \mathbb{R}^{p+1}$, the quantity $\sigma(X^{(i)} \cdot \theta)$.

_Python hint:_ you can use `np.exp(x)` to compute the exponentiation of a float (or term-wise exponentiation of an array) `x`.

In [ ]:
def sigmoid(x, theta):
    """
        param x:     np.array of shape p
        param theta: np.array of shape (p+1)
        return: float.
    """
    return None # <-- to complete

**2.c.** Implement a function `pred` that provides, given $x^{(i)} \in \mathbb{R}^p$ and $\theta \in \mathbb{R}^{p+1}$, an estimate $\hat{y}^{(i)}$ of $y^{(i)}$.

In [ ]:
def pred(x, theta):
    """
        param x:     np.array of shape p
        param theta: np.array of shape p+1
        return: integer, 0 or 1.
    """
    return None # <-- to complete

**Question 3:** Implement the error function we want to minimize $E$, that takes $x, y, \theta$ as arguments.

In [ ]:
def E(X, y, theta):
    """
        param X:     np.array of shape N x (p+1)
        param y:     np.array of shape N
        param theta: np.array of shape (p+1)
        return: float
    """
    return None # <-- to complete

As there is no closed form for the optimal $\theta^*$, we will minimize $E$ using a gradient descent. 

**Question 4:** Implement a function `grad_E(X,y,theta)` that returns the gradient (with respect to $\theta$) of $E$ for the current estimation $\theta$ of $E$.

_Hints:_ If you don't remember the expression of the gradient for this problem seen in lectures 4 and 5, you can derive it on a separate sheet of paper. Do not forget the $\frac{1}{n}$ factor.

In [ ]:
def grad_E(X, y, theta):
    """
        param X:     np.array of shape N x (p+1)
        param y:     np.array of shape N
        param theta: np.array of shape (p+1)
        return: np.array of shape (p+1)
    """
    return None # <-- to complete

**Question 5:** Complete the following code in order to implement the gradient descent algorithm to minimize $E$ that returns the final estimation of $\theta$ and the evolution of energy over time. 

The hyper parameters (number of steps, learning rates) are set by default and should not be changed.

The gradient descent will use the following rules:
- If $E(\theta^{(t+1)}) > E(\theta^{(t)})$, then we devide the learning rate (stored in `eta`) by $2$.
- If $\left|E(\theta^{(t+1)}) - E(\theta^{(t)})\right|$ is smaller than a fixed value (stored in `stopping_criterion`), we stop the algorithm.

_Python hints:_ 
- You can use `np.abs(x)` to obtain the absolute value of a float `x`.

- You can use `break` to exit a `for` loop. For example, 

        for t in range(1000): 
            print(t)
            if t == 42:
                break
            
will print `0, 1, ... , 42` (and nothing more).

In [ ]:
def grad_descent(x, y, eta=0.001, nb_max_step=10000, stopping_criterion=0.01, verbose=False):
    """
        param x:                  np.array, input observations, shape N x p
        param y:                  np.array, input labels, shape N (filled with 0 and 1)
        param eta:                float, learning rate.
        param nb_max_step:        int, maximal number of steps done in the gradient descent.
        param stopping_criterion: float, stopping criterion on gradient process.
        
        return: np.array of shape (p+1) (theta), list of float (evolution_of_loss)
        
        Remark: do not change the default values!
    """
    # Storing some useful values
    N, p = x.shape
    # List to store the evolution of E(theta) over steps
    evolution_of_loss = []
    # Store the current value of loss to stop gradient descent
    e = np.inf
    # Initialization of theta (using some hidden function)
    theta = init_theta(p)
    
    ### Perform the gradient descent
    for t in range(nb_max_step):
        
        # Perform a gradient step
        #<-- to complete
        new_e = 0 #<-- to complete, store the loss according to the new parameter theta
        
        # In testing process (below) we print the evolution
        # of the training error (which should decrease)
        if verbose and ((t+1)%10)==0:
            print("Training error after",t+1,"steps:", np.round(new_e,5))
        
        # Stopping criterion:
        #<--to complete
    
    return theta, evolution_of_loss

## Test on a 1D dataset

**Question 5:** test your code with the cell below.

In [ ]:
test_1d_logreg(grad_descent, pred, sigmoid)

## On a 2D dataset

We consider a dataset composed of a training set `(x_train,y_train)` with $N = 280$ points and a test set `x_test` of $N' = 20$ points (along with some unknown labels `y_test`) that you can load using the following cell. The score of a classifier is defined as
$$ \frac{1}{N'} \sum_{i=1}^{N'} \delta_{\mathrm{pred}(x_\mathrm{test}^{(i)}), y_\mathrm{test}^{(i)}} $$
where
$$\delta_{\alpha,\beta} = \begin{cases} 1 \text{ if } \alpha = \beta \\ 0 \text{ otherwise } \end{cases}$$

In [ ]:
x_train, y_train, x_test, y_test = load_2D_set()

**Question 6:** 

**6.a.** What is the training error of the logistic regression on this dataset?

In [ ]:
# Write some code here to answer the question

Write the answer of the question in the cell below.

<-- here -->

**6.b.** What is your classification score?

In [ ]:
# Write some code here to answer the question

Write the answer of the question in the cell below.

<-- here -->